In [ ]:
import os
import sys
import json
import numpy as np
import pandas as pd
from datetime import datetime
from collections import Counter
import matplotlib.pyplot as plt

In [ ]:
config = '../configs/config_shiva.json'
config = json.load(open(config))

In [ ]:
data_dir = config['data_dir']
raw_dir = "{}/{}".format(data_dir, config['raw_data_dir'])
csv_dir = "{}/{}".format(data_dir, config['raw_data_csv'])
ltsf = "{}/ltsf".format(data_dir)

In [ ]:
file = os.listdir(ltsf)[0]
file

In [ ]:
df = pd.read_csv("{}/{}".format(csv_dir, file))

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
for col in df.columns:
    if col != 'date':
        plt.figure(figsize=(11, 3)) ; plt.plot(df[col]) ; plt.show()

(batch_size * num_stocks * seq_len)

In [ ]:
16 * 1500 * 360

In [ ]:
1200/1400/700